## 1. Setup & Data Preparation

In [23]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim 
from torch.optim.lr_scheduler import StepLR
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision import models
from PIL import Image
from tqdm import tqdm
import pandas as pd

In [3]:
data_dir = "UTKFace"
model_path = "models"

In [4]:
# Define the dataset class
class UTKFaceDataset(Dataset):
    def __init__(self, root_dir, transforms=None):
        self.root_dir = root_dir
        self.transform = transforms
        self.image_files = [f for f in os.listdir(root_dir) if f.endswith(".jpg")]
        
    def __len__(self):
        return len(self.image_files)
    
    def __getitem__(self, index):
        image_name = self.image_files[index]
        image_path = os.path.join(self.root_dir, image_name)
        image = Image.open(image_path).convert('RGB')
        
        # Extract age and gender from the filename
        age = int(image_name.split('_')[0])
        gender = int(image_name.split('_')[1])
        
        if self.transform:
            image = self.transform(image)
        return image, torch.tensor([age], dtype=torch.float32), torch.tensor([gender], dtype=torch.long)

# Define the transformation

train_transforms  = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

test_transforms = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# Split the dataset into training and testing sets
train_dataset, test_dataset = torch.utils.data.random_split(
    UTKFaceDataset(data_dir, transforms=test_transforms),
    [int(0.8 * len(os.listdir(data_dir))), int(0.2 * len(os.listdir(data_dir)))],
    generator=torch.Generator().manual_seed(42)
)

test_dataset.transform = test_transforms
train_dataset.transform = train_transforms
# over sample the training dataset to balance the classes
class_counts = pd.Series([int(f.split('_')[1]) for f in os.listdir(data_dir) if f.endswith('.jpg')]).value_counts()
max_class_count = class_counts.max()
repeat_count = max(1, max_class_count // len(train_dataset))
train_dataset = torch.utils.data.ConcatDataset([
    UTKFaceDataset(data_dir, transforms=train_transforms) for _ in range(repeat_count)
])


# Dataloaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)



### Gender distribution in training and testing dataset

- ### 0: Male
- ### 1: Female

In [5]:
from tqdm import tqdm
training_female_count = 0
training_male_count = 0

for i in tqdm(range(len(train_dataset))):
    label = train_dataset[i][2]
    if torch.equal(label, torch.tensor([1])):
        training_female_count += 1
    elif torch.equal(label, torch.tensor([0])):
        training_male_count += 1



100%|██████████| 20855/20855 [01:59<00:00, 174.10it/s]


In [6]:
testing_female_count = 0
testing_male_count = 0

for i in tqdm(range(len(test_dataset))):
    label = test_dataset[i][2]
    if torch.equal(label, torch.tensor([1])):
        testing_female_count += 1 
    elif torch.equal(label, torch.tensor([0])):
        testing_male_count += 1

100%|██████████| 4171/4171 [00:23<00:00, 179.86it/s]


In [7]:
class_counts = {
    "Female": training_female_count,
    "Male": training_male_count
}
print("Class counts in training set:")
print(class_counts)
print("Total number of training images:", len(train_dataset))

print("__" * 50)
test_class_counts = {
    "Female": testing_female_count,
    "Male": testing_male_count
}
print("Class counts in test set:")
print(test_class_counts)
print("Total number of test images:", len(test_dataset))

Class counts in training set:
{'Female': 10528, 'Male': 10327}
Total number of training images: 20855
____________________________________________________________________________________________________
Class counts in test set:
{'Female': 2113, 'Male': 2058}
Total number of test images: 4171


## Training and testing models

### Defined training and evaluating functions

In [5]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

Using device: cuda


In [6]:
# Evaluate function: Calculate accuracy on the test set
def evaluate_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images in tqdm(test_loader):
            inputs, _, labels = images
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels.squeeze()).sum().item()
    
    accuracy = correct / total
    print(f"Test Accuracy: {accuracy:.4f}")
    return accuracy


# Training function
def train_model(model, train_loader, criterion, optimizer, lr_scheduler=None ,num_epochs=10):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        accuracy = 0.0
        for images in tqdm(train_loader):
            inputs, _, labels = images
            inputs, labels = inputs.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels.squeeze())
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            accuracy += (outputs.argmax(dim=1) == labels.squeeze()).sum().item()
        if lr_scheduler:
            lr_scheduler.step()
        accuracy /= len(train_loader.dataset)
        avg_loss = running_loss / len(train_loader)
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}, Accuracy: {accuracy:.4f}")  
        evaluate_model(model, test_loader)


        

### Pretrained resnet model

  self.classifier = nn.Sequential(
            nn.Linear(512 * 4 * 4, 4096), 
            nn.ReLU(),
            nn.BatchNorm1d(4096),
            nn.Dropout(0.5),
            nn.Linear(4096, 1024),
            nn.ReLU(),
            nn.BatchNorm1d(1024),
            nn.Dropout(0.5),
            nn.Linear(1024, num_classes)
        )

In [ ]:
from torchvision.models import resnet50

class ResNet50_f(nn.Module):
    def __init__(self, num_classes=2):
        super().__init__()
        base_model = resnet50(pretrained=True)
        self.features = nn.Sequential(*list(base_model.children())[:-1])  # remove final fc

        self.classifier = nn.Sequential(
            nn.Linear(2048, 4096),
            nn.ReLU(),
            nn.BatchNorm1d(4096),
            nn.Dropout(0.5),

            nn.Linear(4096, 1024),
            nn.ReLU(),
            nn.BatchNorm1d(1024),
            nn.Dropout(0.5),
            
            nn.Linear(1024, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)  # flatten
        x = self.classifier(x)
        return x


In [8]:
resnet50_model = ResNet50_f(num_classes=2).to(device)

# Freeze the feature extractor layers
for param in resnet50_model.features.parameters():
    param.requires_grad = False
    

# Define loss function and optimizer
trainable_params = [param for param in resnet50_model.parameters() if param.requires_grad]

optimizer = optim.Adam(trainable_params, lr=0.0001)
criterion = nn.CrossEntropyLoss()
# Learning rate scheduler
lr_scheduler = StepLR(optimizer, step_size=10, gamma=0.1)


d:\Anaconda3\envs\nlp_env\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
d:\Anaconda3\envs\nlp_env\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [14]:
# Train the resnet50 model
train_model(resnet50_model, train_loader, criterion, optimizer, lr_scheduler, num_epochs=35)

100%|██████████| 652/652 [03:12<00:00,  3.39it/s]


Epoch [1/35], Loss: 0.4492, Accuracy: 0.7833


100%|██████████| 131/131 [00:39<00:00,  3.35it/s]


Test Accuracy: 0.8243


100%|██████████| 652/652 [03:04<00:00,  3.53it/s]


...

Test Accuracy: 0.8772


100%|██████████| 652/652 [02:55<00:00,  3.71it/s]


Epoch [35/35], Loss: 0.2939, Accuracy: 0.8750


100%|██████████| 131/131 [00:33<00:00,  3.87it/s]

Test Accuracy: 0.8775


In [15]:
# Save the resnet50 model
torch.save(resnet50_model.state_dict(), os.path.join(model_path, "resnet50_f_gender_classifier.pth"))

### Pretrained VGG_f model

In [9]:
class VGG_f(nn.Module):
    def __init__(self, num_classes=2):
        super().__init__()
        base_model = models.vgg16(pretrained=True)
        self.features = base_model.features  # Convolutional part

        self.classifier = nn.Sequential(
            nn.Linear(512 * 4 * 4, 4096), 
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(4096, 1024),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(1024, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x


In [10]:
# Initialize the VGG model
vgg_model = VGG_f(num_classes=2).to(device)
# Freeze the feature extractor layers
for param in vgg_model.features.parameters():
    param.requires_grad = False
    
# Define loss function and optimizer
trainable_params = [param for param in vgg_model.parameters() if param.requires_grad]

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(trainable_params, lr=0.0001)
lr_scheduler = StepLR(optimizer, step_size=10, gamma=0.1)

d:\Anaconda3\envs\nlp_env\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [18]:
# Train the vgg model
train_model(vgg_model, train_loader, criterion, optimizer, lr_scheduler, num_epochs=35)

100%|██████████| 652/652 [02:55<00:00,  3.71it/s]


Epoch [1/35], Loss: 0.3917, Accuracy: 0.8244


100%|██████████| 131/131 [00:33<00:00,  3.88it/s]


Test Accuracy: 0.8593


100%|██████████| 652/652 [02:56<00:00,  3.70it/s]


...

Test Accuracy: 0.9883


100%|██████████| 652/652 [02:56<00:00,  3.69it/s]


Epoch [35/35], Loss: 0.0390, Accuracy: 0.9866


100%|██████████| 131/131 [00:33<00:00,  3.86it/s]

Test Accuracy: 0.9885


In [19]:
# Save the vgg model
torch.save(vgg_model.state_dict(), os.path.join(model_path, "vgg_f_gender_classifier.pth"))

## Pretrained models with some modules added

In [11]:
# Define the Channel Attention module
class ChannelAttention(nn.Module):
    def __init__(self, in_planes, ratio=8):
        super(ChannelAttention, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.max_pool = nn.AdaptiveAvgPool2d(1)
        
        self.fc = nn.Sequential(
            nn.Conv2d(in_planes, in_planes // ratio, 1, bias=False),
            nn.ReLU(),
            nn.Conv2d(in_planes // ratio, in_planes, 1, bias=False)
        )
        
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        avg_out = self.fc(self.avg_pool(x))
        max_out = self.fc(self.max_pool(x))
        out = avg_out + max_out
        
        return self.sigmoid(out) * x 

# Define the Spatial Attention module
class SpatialAttention(nn.Module):
    def __init__(self, kernel_size=7):
        super(SpatialAttention, self).__init__()
        padding = kernel_size // 2
        self.conv = nn.Conv2d(2, 1, kernel_size, padding=padding, bias=False)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        avg_out = torch.mean(x, dim=1, keepdim=True)
        max_out = torch.max(x, dim=1, keepdim=True).values
        x_cat = torch.cat([avg_out, max_out], dim=1)
        return self.sigmoid(self.conv(x_cat)) * x
    
# Define the CBAM module
class CBAM(nn.Module):
    def __init__(self, in_planes, ratio=8, kernel_size=7):
        super(CBAM, self).__init__()
        self.channel_attention = ChannelAttention(in_planes, ratio)
        self.spatial_attention = SpatialAttention(kernel_size)
        
    def forward(self, x):
        x = self.channel_attention(x)
        x = self.spatial_attention(x)
        return x

### Pretrained resnet with CBAM modules

In [17]:
class GhostModule(nn.Module):
    def __init__(self, in_channels, out_channels, ratio=2, dw_size=3):
        super(GhostModule, self).__init__()
        self.intrinsic_channels = out_channels // ratio
        self.primary_conv = nn.Conv2d(in_channels, self.intrinsic_channels, 1, bias=False)
        self.ghost_conv = nn.Conv2d(self.intrinsic_channels, self.intrinsic_channels, dw_size, padding=dw_size//2, groups=self.intrinsic_channels, bias=False)
        self.bn = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        intrinsic = self.primary_conv(x)
        ghost = self.ghost_conv(intrinsic)
        out = torch.cat([intrinsic, ghost], dim=1)
        out = self.bn(out)
        return self.relu(out)

In [ ]:
# Build the modified ResNet model with CBAM
class ResNetCBAM(nn.Module):
    def __init__(self, num_classes=2):
        super(ResNetCBAM, self).__init__()
        base_model = resnet50(pretrained=True)
        self.features = nn.Sequential(*list(base_model.children())[:-1])  # remove final fc
        
        self.cbam = CBAM(2048, ratio=8, kernel_size=7)
        self.ghost_module = GhostModule(2048, 512, ratio=2, dw_size=3)
        
        self.classifier = nn.Sequential(
            nn.Linear(2048, 4096),
            nn.ReLU(),
            nn.BatchNorm1d(4096),
            nn.Dropout(0.5),

            nn.Linear(4096, 1024),
            nn.ReLU(),
            nn.BatchNorm1d(1024),
            nn.Dropout(0.5),
            
            nn.Linear(1024, num_classes)
        )
        
    
        
    def forward(self, x):
        x = self.features(x)  # [batch, 2048, 1, 1]
        x = self.cbam(x)      # [batch, 2048, 1, 1]
        x = self.ghost_module(x)  # [batch, 512, 1, 1]
        x = x.view(x.size(0), -1) # flatten to [batch, 512]
        x = self.classifier(x)
        return x
        

In [19]:
# # Initialize the modified model
cbam_resnet = ResNetCBAM(num_classes=2)
cbam_resnet.to(device)

# Freeze the convolutional layers
for param in cbam_resnet.parameters():
    param.requires_grad = False

# Unfreeze CBAM and fc layers
for param in cbam_resnet.cbam.parameters():
    param.requires_grad = True
for param in cbam_resnet.ghost_module.parameters():
    param.requires_grad = True
for param in cbam_resnet.classifier.parameters():
    param.requires_grad = True

# Optimizer with only trainable parameters
trainable_params = [p for p in cbam_resnet.parameters() if p.requires_grad]
cbam_optimizer = optim.Adam(trainable_params, lr=0.0001, weight_decay=1e-4)

# Loss function
criterion = nn.CrossEntropyLoss()

# Learning rate scheduler
cbam_lr_scheduler = StepLR(cbam_optimizer, step_size=10, gamma=0.1)


d:\Anaconda3\envs\nlp_env\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
d:\Anaconda3\envs\nlp_env\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [24]:
# Train the modified model
train_model(cbam_resnet, train_loader, criterion, cbam_optimizer, cbam_lr_scheduler, num_epochs=35)

100%|██████████| 652/652 [02:57<00:00,  3.67it/s]


Epoch [1/35], Loss: 0.4308, Accuracy: 0.7960


100%|██████████| 131/131 [00:35<00:00,  3.70it/s]


Test Accuracy: 0.8327


100%|██████████| 652/652 [02:54<00:00,  3.74it/s]


...

Test Accuracy: 0.9099


100%|██████████| 652/652 [02:51<00:00,  3.80it/s]


Epoch [35/35], Loss: 0.2292, Accuracy: 0.9040


100%|██████████| 131/131 [00:33<00:00,  3.96it/s]

Test Accuracy: 0.9099


In [25]:
torch.save(cbam_resnet.state_dict(), os.path.join(model_path, "cbam_resnet18_gender_classifier.pth"))

### Pretrained VGG with CBAM 

In [46]:
# Build the modified VGG model with CBAM and GhostNet
class VGGCBAM(nn.Module):
    def __init__(self, num_classes=2):
        super(VGGCBAM, self).__init__()
        base_model = models.vgg16(pretrained=True)
        self.features = base_model.features  # Convolutional part
        
        self.cbam = CBAM(512, ratio=8, kernel_size=7)
    
        self.classifier = nn.Sequential(
            nn.Linear(512 * 4 * 4, 4096), 
            nn.ReLU(),
            nn.BatchNorm1d(4096),
            nn.Dropout(0.5),
            nn.Linear(4096, 1024),
            nn.ReLU(),
            nn.BatchNorm1d(1024),
            nn.Dropout(0.5),
            nn.Linear(1024, num_classes)
        )
        
    def forward(self, x):
        x = self.features(x)  # [batch, 512, 4, 4]
        x = self.cbam(x)      # [batch, 512, 4, 4]
        x = x.view(x.size(0), -1) # flatten to [batch, 512 * 4 * 4]
        x = self.classifier(x)
        return x

In [47]:
# Initialize the VGGGhostCBAM model
vgg_cbam = VGGCBAM(num_classes=2)
vgg_cbam.to(device)
# Freeze the convolutional layers
for param in vgg_cbam.parameters():
    param.requires_grad = False
# Unfreeze CBAM and fc layers
for param in vgg_cbam.cbam.parameters():
    param.requires_grad = True
for param in vgg_cbam.classifier.parameters():
    param.requires_grad = True

# Optimizer with only trainable parameters
trainable_params = [p for p in vgg_cbam.parameters() if p.requires_grad]
vgg_cbam_optimizer = optim.Adam(trainable_params, lr=0.0001, weight_decay=1e-4)
# Loss function
criterion = nn.CrossEntropyLoss()
# Learning rate scheduler
vgg_cbam_lr_scheduler = StepLR(vgg_cbam_optimizer, step_size=10, gamma=0.1)


d:\Anaconda3\envs\nlp_env\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
d:\Anaconda3\envs\nlp_env\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [51]:
# Train the vgg modified model
train_model(vgg_cbam, train_loader, criterion, vgg_cbam_optimizer, vgg_cbam_lr_scheduler, num_epochs=35)


100%|██████████| 652/652 [03:06<00:00,  3.49it/s]


Epoch [1/35], Loss: 0.4407, Accuracy: 0.8094


100%|██████████| 131/131 [00:35<00:00,  3.66it/s]


Test Accuracy: 0.8655


100%|██████████| 652/652 [03:04<00:00,  3.54it/s]


...

Test Accuracy: 0.9890


100%|██████████| 652/652 [03:00<00:00,  3.61it/s]


Epoch [35/35], Loss: 0.0345, Accuracy: 0.9880


100%|██████████| 131/131 [00:33<00:00,  3.85it/s]

Test Accuracy: 0.9890


In [52]:
# Save the vgg modified model
torch.save(vgg_cbam.state_dict(), os.path.join(model_path, "vgg_ghost_cbam_gender_classifier.pth"))

### Pretrained VGG with SE and SA Blocks

In [53]:
# Define the SEBlock module
class SEBlock(nn.Module):
    def __init__(self, in_channels, reduction=16):
        super(SEBlock, self).__init__()
        self.global_avg_pool = nn.AdaptiveAvgPool2d(1) # Calculate the average weight of each channel
        self.fc = nn.Sequential(
            nn.Linear(in_channels, in_channels // reduction, bias=False),
            nn.ReLU(inplace=True),
            nn.Linear(in_channels // reduction, in_channels, bias=False),
            nn.Sigmoid()
        )
        
    def forward(self, x):
        b, c, _, _ = x.size() # Batch size and number of channels
        y = self.global_avg_pool(x).view(b, c) # [B, C]
        y = self.fc(y).view(b, c, 1, 1) # [B, C, 1, 1]
        
        return x * y.expand_as(x)  # Scaled channel wise
        

In [54]:
# Build the modified VGG model with Squeeze-and-Excitation (SE) and SA modules
class VGGSEandSA(nn.Module):
    def __init__(self, num_classes=2):
        super(VGGSEandSA, self).__init__()
        base_model = models.vgg16(pretrained=True)
        self.features = base_model.features  # Convolutional part
        
        self.se_block = SEBlock(512, reduction=16)
        self.spatial_attention = SpatialAttention(kernel_size=7)

        self.classifier = nn.Sequential(
            nn.Linear(512 * 4 * 4, 4096), 
            nn.ReLU(),
            nn.BatchNorm1d(4096),
            nn.Dropout(0.5),
            nn.Linear(4096, 1024),
            nn.ReLU(),
            nn.BatchNorm1d(1024),
            nn.Dropout(0.5),
            nn.Linear(1024, num_classes)
        )
        
    def forward(self, x):
        x = self.features(x)  # [batch, 512, 4, 4]
        x = self.se_block(x)  # [batch, 512, 4, 4]
        x = self.spatial_attention(x)  # [batch, 512, 4, 4]
        x = x.view(x.size(0), -1) # flatten to [batch, 512 * 4 * 4]
        x = self.classifier(x)
        return x

In [55]:
# Initialize the modified VGGSEandSA
vgg_se_sa = VGGSEandSA(num_classes=2).to(device)

# Freeze the convolutional layers
for param in vgg_se_sa.parameters():
    param.requires_grad = False

# Unfreeze SE Block, Spatial Attention Block and fc layers
for param in vgg_se_sa.se_block.parameters():
    param.requires_grad = True

for param in vgg_se_sa.spatial_attention.parameters():
    param.requires_grad = True 
    
for param in vgg_se_sa.classifier.parameters():
    param.requires_grad = True
    
# Optimizer with only trainable parameters
trainable_params = [p for p in vgg_se_sa.parameters() if p.requires_grad]
vgg_se_sa_optimizer = optim.Adam(trainable_params, lr=0.0001, weight_decay=1e-4)
# Loss function
criterion = nn.CrossEntropyLoss()
# Learning rate scheduler
vgg_se_sa_lr_scheduler = StepLR(vgg_se_sa_optimizer, step_size=5, gamma=0.2)

d:\Anaconda3\envs\nlp_env\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
d:\Anaconda3\envs\nlp_env\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [56]:
# Train the vgg_se_sa model 
train_model(vgg_se_sa, train_loader, criterion, vgg_se_sa_optimizer, vgg_se_sa_lr_scheduler, num_epochs=35)

100%|██████████| 652/652 [03:16<00:00,  3.32it/s]


Epoch [1/35], Loss: 0.4386, Accuracy: 0.8072


100%|██████████| 131/131 [00:36<00:00,  3.62it/s]


Test Accuracy: 0.8732


100%|██████████| 652/652 [03:15<00:00,  3.34it/s]


...

Epoch [26/35], Loss: 0.0680, Accuracy: 0.9762


100%|██████████| 131/131 [00:33<00:00,  3.87it/s]


Test Accuracy: 0.9784


 97%|█████████▋| 632/652 [02:55<00:05,  3.60it/s]


KeyboardInterrupt: 

### The training took too long and the accuracy seemed can not be improved and i interrupted it and save the model

In [57]:
# Save the vgg_se_sa model
torch.save(vgg_se_sa.state_dict(), os.path.join(model_path, "vgg_se_sa_gender_classifier.pth"))